<a href="https://colab.research.google.com/github/TMUITLab/EAFR/blob/master/EA13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd '/content'
!git clone https://github.com/1049451037/GCN-Align
!git clone https://github.com/MaoXinn/RREA

/content
Cloning into 'GCN-Align'...
remote: Enumerating objects: 53, done.
remote: Total 53 (delta 0), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (53/53), done.
Cloning into 'RREA'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 32 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [27]:
import warnings
warnings.filterwarnings('ignore')

from importlib.machinery import SourceFileLoader
from google.colab import files

layer = SourceFileLoader("layer", "/content/RREA/CIKM/layer.py").load_module()
utils = SourceFileLoader("utils", "/content/RREA/CIKM/utils.py").load_module()
CSLS = SourceFileLoader("CSLS", "/content/RREA/CIKM/CSLS.py").load_module()
import tensorflow as tf
import os
import time
import random
import keras
import pickle
from tqdm import *
import numpy as np
from utils import *
from CSLS import *
import tensorflow as tf
import keras.backend as K
from keras.layers import *
from layer import NR_GraphAttention

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [14]:
lang = 'zh'
radj,train_pair,dev_pair,adj_matrix,r_index,r_val,adj_features,rel_features,nadj,char_size = load_data('/content/GCN-Align/data/%s_en/'%lang,train_ratio=0.30)
train_pair_main=train_pair
adj_matrix = np.stack(adj_matrix.nonzero(),axis = 1)
rel_matrix,rel_val = np.stack(rel_features.nonzero(),axis = 1),rel_features.data
ent_matrix,ent_val = np.stack(adj_features.nonzero(),axis = 1),adj_features.data

38960 3024


In [13]:
single_triple_set = set()
rel_size = rel_features.shape[1] 
degree = np.zeros([adj_features.shape[0],3])
for triple in radj:
    single_triple_set.add((triple[0],triple[1]))
    if triple[2] < rel_size //2:
        degree[triple[0],0] += 1
        degree[triple[1],0] += 1
        degree[triple[0],2] += 1
        degree[triple[1],1] += 1

dev_pair_degrees = np.zeros([2,len(dev_pair),3])
for i in range(len(dev_pair)):
    dev_pair_degrees[0,i,0] = degree[dev_pair[i][0],0]
    dev_pair_degrees[0,i,1] = degree[dev_pair[i][0],1]
    dev_pair_degrees[0,i,2] = degree[dev_pair[i][0],2]
    dev_pair_degrees[1,i,0] = degree[dev_pair[i][1],0]
    dev_pair_degrees[1,i,1] = degree[dev_pair[i][1],1]
    dev_pair_degrees[1,i,2] = degree[dev_pair[i][1],2]
map = dict()
for t in train_pair:
    map[t[0]] = t[1]
    map[t[1]] = t[0]

In [ ]:
entity1, rel1, triples1 = load_triples('/content/GCN-Align/data/%s_en/'%lang + 'triples_1')
num_entity_1 = len(entity1)
num_rel_1 = len(rel1)

In [30]:
layer = SourceFileLoader("layer", "/content/RREA/CIKM/layer.py").load_module()

from layer import NR_GraphAttention
tf.keras.backend.clear_session()

node_size = adj_features.shape[0]
rel_size = rel_features.shape[1]
triple_size = len(adj_matrix)
batch_size = node_size
datas = []
# e stand for entity ,r stand for relation , E stand for encoder(e), c stand for character embedding , R stand for encoder(r)
run_name = 'eER'


class TokenEmbedding(keras.layers.Embedding):
    """Embedding layer with weights returned."""

    def compute_output_shape(self, input_shape):
        return self.input_dim, self.output_dim

    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, inputs):
        return self.embeddings
    
def get_embedding():
    inputs = [adj_matrix,r_index,r_val,rel_matrix,ent_matrix,radj,nadj]
    inputs = [np.expand_dims(item,axis=0) for item in inputs]
    return get_emb.predict_on_batch(inputs)


def test(wrank = None):
    vec = get_embedding()
    return  get_hits(vec,dev_pair,wrank=wrank)

def CSLS_test(thread_number = 16, csls=10,accurate = True):
    vec = get_embedding()
    trest_set_1 = [e1 for e1, e2 in dev_pair]
    trest_set_2 = [e2 for e1, e2 in dev_pair]
    Lvec = np.array([vec[e1] for e1, e2 in dev_pair])
    Rvec = np.array([vec[e2] for e1, e2 in dev_pair])
    Lvec = Lvec / np.linalg.norm(Lvec,axis=-1,keepdims=True)
    Rvec = Rvec / np.linalg.norm(Rvec,axis=-1,keepdims=True)
    lprec,ldata = eval_alignment_by_sim_mat(Lvec, Rvec, [1, 5, 10], thread_number, csls=csls, accurate=accurate)
    rprec,rdata = eval_alignment_by_sim_mat(Rvec, Lvec, [1, 5, 10], thread_number, csls=csls, accurate=accurate)
    results = np.zeros([2,len(dev_pair)])

    for i in range(len(dev_pair)):
        if((i,i) in lprec):
            results[0,i] = 1
        if((i,i) in rprec):
            results[1,i] = 1
    datas.append((results,ldata,rdata))
    save_datas()
    return None
def save_datas():
    with open('/content/RREA/'+lang+"_"+run_name+'.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump([dev_pair_degrees,datas], f)

def load_datas():
    with open('/content/RREA/'+lang+"_"+run_name+'.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
        mydata =pickle.load([dev_pair_degrees,datas], f)
    return mydata;

def get_train_set(batch_size = batch_size):
    negative_ratio =  batch_size // len(train_pair) + 1
    train_set = np.reshape(np.repeat(np.expand_dims(train_pair,axis=0),axis=0,repeats=negative_ratio),newshape=(-1,2))
    np.random.shuffle(train_set); train_set = train_set[:batch_size]
    train_set = np.concatenate([train_set,np.random.randint(0,node_size,[train_set.shape[0],4])],axis = -1)
    return train_set

def get_train_set1(batch_size = batch_size):
    train_set = train_pair
    np.random.shuffle(train_set);
    train_set = np.concatenate([train_set,np.random.randint(0,node_size,train_set.shape)],axis = -1)
    return train_set

def get_trgat(node_size,rel_size,node_hidden,rel_hidden,triple_size,n_attn_heads = 2,dropout_rate = 0,gamma = 3,lr = 0.005,depth = 2):
    adj_input = Input(shape=(None,2))
    index_input = Input(shape=(None,2),dtype='int64')
    val_input = Input(shape = (None,))
    rel_adj = Input(shape=(None,2))
    ent_adj = Input(shape=(None,2))
    radj = Input(shape=(None,3),dtype='int64')
    nadj = Input(shape=(None,3))
    
    # ent_emb = TokenEmbedding(node_size,node_hidden,embeddings_initializer="LecunNormal",trainable = True)(val_input) 
    # rel_emb = TokenEmbedding(rel_size,node_hidden,embeddings_initializer="LecunNormal",trainable = True)(val_input)
    # ch_emb = TokenEmbedding(char_size,node_hidden,embeddings_initializer="LecunNormal",trainable = True)(val_input)

    ent_emb = TokenEmbedding(node_size,node_hidden,trainable = True)(val_input) 
    rel_emb = TokenEmbedding(rel_size,node_hidden,trainable = True)(val_input)
    ch_emb = TokenEmbedding(char_size,node_hidden,trainable = True)(val_input)

    # E = TokenEmbedding(node_hidden,node_hidden,trainable = True)(val_input)
    # R = TokenEmbedding(node_hidden,node_hidden,trainable = True)(val_input)
    def ch_avg(tensor,size):
        n_adj = tf.squeeze(tensor[0],axis = 0)
        adj = K.cast(n_adj[:,0:2],dtype = "int64")   
        adj = tf.SparseTensor(indices=adj, values=tf.ones_like(n_adj[:,2],dtype = 'float32') , dense_shape=(node_size,size)) 
        adj = tf.compat.v1.sparse_softmax(adj) 
        l_adj = tf.compat.v1.sparse_tensor_dense_matmul(adj,tensor[1])
        return l_adj

    def avg(tensor,size):
        adj = K.cast(K.squeeze(tensor[0],axis = 0),dtype = "int64")   
        adj = tf.SparseTensor(indices=adj, values=tf.ones_like(adj[:,0],dtype = 'float32'), dense_shape=(node_size,size)) 
        adj = tf.compat.v1.sparse_softmax(adj) 
        l_adj = tf.compat.v1.sparse_tensor_dense_matmul(adj,tensor[1])
        return l_adj

    name_feature = Lambda(ch_avg,arguments={'size':char_size})([nadj,ch_emb])
    rel_feature = Lambda(avg,arguments={'size':rel_size})([rel_adj,rel_emb])
    ent_feature = Lambda(avg,arguments={'size':node_size})([ent_adj,ent_emb])

    encoder = NR_GraphAttention(node_size,activation="relu",
                                       rel_size = rel_size,
                                       depth = 2,
                                       attn_heads=n_attn_heads,
                                       triple_size = triple_size,
                                       attn_heads_reduction='average',   
                                       dropout_rate=dropout_rate)
    
    opt1 = [rel_emb,adj_input,index_input,val_input,radj]
    elements = []
    coefs = []
    for f_name in run_name:
        if f_name == 'e':
            elements += [ent_emb]
            coefs += [-0.5]
        elif f_name == 'E':
            elements += encoder([ent_feature]+opt1)
            coefs += [1.,1.,1.]
        elif f_name == 'R':
            elements += encoder([rel_feature]+opt1)
            coefs += [1.,1.,1.]
        elif f_name == 'c':
            elements += [name_feature]
            coefs += [1.,1.,1.]

    #elements = [ent_emb] +   encoder([rel_feature]+opt1)+encoder([ent_feature]+opt1)
    #elements = [0.3 * ent_emb[:,:node_hidden//2]]+ el[0:3]+el[6:9]+[0.2 * ent_emb[:,node_hidden//2:]] + el[3:6]+el[9:12]
    num_el = elements.__len__()
    weight =  tf.Variable(coefs, trainable=False)
    weight = 5 * tf.math.softmax(weight)
    #weight =  tf.Variable(1 * [-0.2,1.0] + (num_el-2) * [.8], trainable=False)
    print(weight)
    elements = [weight[idx]*el for idx, el in enumerate(elements)]
    out_feature = Concatenate(-1)(elements)
    out_feature = Dropout(dropout_rate)(out_feature)
    
    alignment_input = Input(shape=(None,6),dtype = "int64") 
    #find = Lambda(lambda x:K.gather(reference=x[0],indices=K.cast(K.squeeze(x[1],axis=0), 'int32')))([out_feature,alignment_input])

    I = K.cast(K.squeeze(alignment_input,axis=0),dtype = "int64")
   
    A = K.sum(K.abs(K.gather(out_feature,I[:,0])-K.gather(out_feature,I[:,1])),axis=-1,keepdims=True)
    loss =  K.relu(1 - K.sum(K.abs(K.gather(out_feature,I[:,0])))) +K.relu(1 - K.sum(K.abs(K.gather(out_feature,I[:,1]))))
    for i in range(4):
        B = K.sum(K.abs(K.gather(out_feature,I[:,0])-K.gather(out_feature,I[:,2+i])),axis=-1,keepdims=True)
        C = K.sum(K.abs(K.gather(out_feature,I[:,1])-K.gather(out_feature,I[:,2+i])),axis=-1,keepdims=True)
        loss += K.relu(gamma +A -B) + K.relu(gamma +A -C)

    loss =  tf.compat.v1.reduce_sum(loss,keep_dims=True) / (batch_size)    
   
    inputs = [adj_input,index_input,val_input,rel_adj,ent_adj,radj,nadj]
    train_model = keras.Model(inputs = inputs + [alignment_input],outputs = loss)
    
    train_model.compile(loss=lambda y_true,y_pred: y_pred,optimizer=tf.keras.optimizers.RMSprop(lr=lr))
    
    feature_model = keras.Model(inputs = inputs,outputs = out_feature)
    return train_model,feature_model


In [31]:
model,get_emb = get_trgat(dropout_rate=0.3,node_size=node_size,rel_size=rel_size,n_attn_heads = 1,depth=2,gamma =3,node_hidden=100,rel_hidden = 100,triple_size = triple_size)
model.summary(); 
initial_weights = model.get_weights()

tf.Tensor(
[0.17927486 0.80345416 0.80345416 0.80345416 0.80345416 0.80345416
 0.80345416], shape=(7,), dtype=float32)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 token_embedding (TokenEmbeddin  (38960, 100)        3896000     ['input_3[0][0]']                
 g)                                                                                               
                                                                                                  
 input_5 (InputLayer)           [(None, None, 2)]    0           []                               
                                                                          

In [ ]:
train_pair = train_pair_main
tf.keras.backend.clear_session()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True  
sess = tf.compat.v1.Session(config=config) 

rest_set_1 = [e1 for e1, e2 in dev_pair]
rest_set_2 = [e2 for e1, e2 in dev_pair]
np.random.shuffle(rest_set_1)
np.random.shuffle(rest_set_2)

epoch = 1000
for turn in range(5):
    print("iteration %d start."%turn)
    for i in trange(epoch):
        cur_epoch = i
        train_set = get_train_set()
        inputs = [adj_matrix,r_index,r_val,rel_matrix,ent_matrix,radj,nadj,train_set]
        inputs = [np.expand_dims(item,axis=0) for item in inputs]
        model.train_on_batch(inputs,np.zeros((1,1)))
        #CSLS_test()
        if i%100 == 99:
            CSLS_test()     
    
    #CSLS_test()       
    new_pair = []
    
    vec = get_embedding()
    Lvec = np.array([vec[e] for e in rest_set_1])
    Rvec = np.array([vec[e] for e in rest_set_2])
    Lvec = Lvec / np.linalg.norm(Lvec,axis=-1,keepdims=True)
    Rvec = Rvec / np.linalg.norm(Rvec,axis=-1,keepdims=True)
    A,_ = eval_alignment_by_sim_mat(Lvec, Rvec, [1, 5, 10], 16,10,True,False)
    B,_ = eval_alignment_by_sim_mat(Rvec, Lvec,[1, 5, 10], 16,10,True,False)
    A = sorted(list(A)); B = sorted(list(B))
    for a,b in A:
        if  B[b][1] == a:
            new_pair.append([rest_set_1[a],rest_set_2[b]])
    print("generate new semi-pairs: %d." % len(new_pair))
    train_pair = np.concatenate([train_pair,np.array(new_pair)],axis = 0)
    for e1,e2 in new_pair:
        if e1 in rest_set_1:
            rest_set_1.remove(e1) 
        
    for e1,e2 in new_pair:
        if e2 in rest_set_2:
            rest_set_2.remove(e2) 

files.download('/content/RREA/'+lang+"_"+run_name+'.pkl') 

iteration 0 start.


 10%|▉         | 99/1000 [02:39<23:01,  1.53s/it]

accurate results: hits@[1, 5, 10] = [64.89 86.83 91.42], mr = 13.550, mrr = 0.745, time = 17.966 s 
accurate results: hits@[1, 5, 10] = [63.49 86.09 91.2 ], mr = 12.979, mrr = 0.735, time = 17.879 s 


 20%|█▉        | 199/1000 [05:50<20:41,  1.55s/it]

accurate results: hits@[1, 5, 10] = [69.75 88.81 92.65], mr = 15.638, mrr = 0.781, time = 18.402 s 
accurate results: hits@[1, 5, 10] = [68.67 88.5  92.67], mr = 14.426, mrr = 0.774, time = 18.069 s 


...

 60%|█████▉    | 599/1000 [18:47<09:59,  1.50s/it]

accurate results: hits@[1, 5, 10] = [80.53 91.72 93.9 ], mr = 29.122, mrr = 0.856, time = 19.872 s 
accurate results: hits@[1, 5, 10] = [80.52 91.72 94.01], mr = 27.378, mrr = 0.856, time = 20.175 s 


 70%|██████▉   | 699/1000 [22:00<07:32,  1.50s/it]

accurate results: hits@[1, 5, 10] = [80.51 91.66 93.87], mr = 30.628, mrr = 0.856, time = 19.239 s 
accurate results: hits@[1, 5, 10] = [80.36 91.51 93.98], mr = 28.686, mrr = 0.855, time = 19.173 s 


 79%|███████▉  | 791/1000 [25:03<05:19,  1.53s/it]

In [23]:
CSLS_test()     

accurate results: hits@[1, 5, 10] = [65.47 86.73 91.39], mr = 13.934, mrr = 0.749, time = 48.036 s 
number of edges is 363526.6666666414
